<a href="https://colab.research.google.com/github/Neel7317/Hyper_Optimization_Techniques/blob/main/RandomSearchCV%26GrideSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [14]:
df=pd.read_csv('diabetes.csv')

In [15]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
#fill 0 with mean of that columns
import numpy as np


In [21]:
df['Insulin']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['SkinThickness']=np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])

In [22]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train,X_test,y_train,y_test=train_test_split(df.drop('Outcome',axis=1),df['Outcome'],test_size=0.2,random_state=2)

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rf=RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=10,min_samples_split=3,min_samples_leaf=2)

In [27]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [28]:
y_pred=rf.predict(X_test)

In [29]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [30]:
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[91 18]
 [16 29]]
0.7792207792207793
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       109
           1       0.62      0.64      0.63        45

    accuracy                           0.78       154
   macro avg       0.73      0.74      0.74       154
weighted avg       0.78      0.78      0.78       154



The main parameters used by a Random Forest Classifier are:

criterion = the function used to evaluate the quality of a split.

max_depth = maximum number of levels allowed in each tree.

max_features = maximum number of features considered when splitting a node.

min_samples_leaf = minimum number of samples which can be stored in a tree leaf.

min_samples_split = minimum number of samples necessary in a node to cause node splitting.

n_estimators = number of trees in the ensamble.

In [31]:
#RandomSaerch CV implementation...

In [32]:
### Manual Hyperparameter Tuning
model=RandomForestClassifier(n_estimators=300,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[95 14]
 [19 26]]
0.7857142857142857
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       109
           1       0.65      0.58      0.61        45

    accuracy                           0.79       154
   macro avg       0.74      0.72      0.73       154
weighted avg       0.78      0.79      0.78       154



In [33]:
from sklearn.model_selection import RandomizedSearchCV

In [36]:
#For n_estimator for no of trees in randomforest
n_estimators=[int(x) for x in np.linspace(100,1000,num=10)]
#for criterion 
criterion=['gini','entropy','None']
#max feature
max_feature=["auto","sqrt","log2"]
#max depth
max_depth=[int(x) for x in np.linspace(10,500,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]


In [42]:
#Random gride
random_gride={
    'n_estimators':n_estimators,
    'criterion':criterion,
    'max_features':max_feature,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf
}

In [43]:
rdomsearch=RandomizedSearchCV(estimator=rf,param_distributions=random_gride,n_iter=100,cv=3,verbose=2)

In [44]:
rdomsearch.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=1000, min_samples_split=14, min_samples_leaf=6, max_features=sqrt, max_depth=336, criterion=None 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
KeyError: 'None'

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  n_estimators=1000, min_samples_split=14, min_samples_leaf=6, max_features=sqrt, max_depth=336, criterion=None, total=   0.4s
[CV] n_estimators=1000, min_samples_split=14, min_samples_leaf=6, max_features=sqrt, max_depth=336, criterion=None 


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
KeyError: 'None'

  FitFailedWarning)


[CV]  n_estimators=1000, min_samples_split=14, min_samples_leaf=6, max_features=sqrt, max_depth=336, criterion=None, total=   0.4s
[CV] n_estimators=1000, min_samples_split=14, min_samples_leaf=6, max_features=sqrt, max_depth=336, criterion=None 


...

[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=8, max_features=log2, max_depth=227, criterion=None, total=   0.3s
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=8, max_features=log2, max_depth=227, criterion=entropy 
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=8, max_features=log2, max_depth=227, criterion=entropy, total=   0.6s
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=8, max_features=log2, max_depth=227, criterion=entropy 
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=8, max_features=log2, max_depth=227, criterion=entropy, total=   0.6s
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=8, max_features=log2, max_depth=227, criterion=entropy 
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=8, max_features=log2, max_depth=227, criterion=entropy, total=   0.6s
[CV] n_estimators=1000, min_samples_split=14, min_samples_leaf=2, max_features=sqrt, max_depth=227, criterion=entropy 
[CV]

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
KeyError: 'None'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
KeyError: 'None'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
KeyError: 'None'

  FitFailedWarning)


[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=log2, max_depth=445, criterion=None, total=   0.1s
[CV] n_estimators=900, min_samples_split=10, min_samples_leaf=2, max_features=log2, max_depth=227, criterion=entropy 
[CV]  n_estimators=900, min_samples_split=10, min_samples_leaf=2, max_features=log2, max_depth=227, criterion=entropy, total=   1.9s
[CV] n_estimators=900, min_samples_split=10, min_samples_leaf=2, max_features=log2, max_depth=227, criterion=entropy 
[CV]  n_estimators=900, min_samples_split=10, min_samples_leaf=2, max_features=log2, max_depth=227, criterion=entropy, total=   1.9s
[CV] n_estimators=900, min_samples_split=10, min_samples_leaf=2, max_features=log2, max_depth=227, criterion=entropy 
[CV]  n_estimators=900, min_samples_split=10, min_samples_leaf=2, max_features=log2, max_depth=227, criterion=entropy, total=   1.9s
[CV] n_estimators=100, min_samples_split=10, min_samples_leaf=6, max_features=log2, max_depth=173, criterion=gini 
[CV

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  4.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=10,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=2,
                                                    min_samples_split=3,
                                                    min_weight_fraction_leaf=0.0,
                 

In [45]:
rdomsearch.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=173, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=900,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [46]:
rdomsearch.best_params_

{'criterion': 'entropy',
 'max_depth': 173,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 900}

In [48]:
best_random_grid=rdomsearch.best_estimator_

In [50]:
best_random_grid

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=173, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=900,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
y_test=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[102   3]
 [ 12  37]]
0.9025974025974026
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       105
           1       0.93      0.76      0.83        49

    accuracy                           0.90       154
   macro avg       0.91      0.86      0.88       154
weighted avg       0.90      0.90      0.90       154



In [56]:
a=X_train.iloc[:1,:]


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
602,1,124.0,74,36.0,30.5,27.8,0.1,30


In [58]:
y_train.iloc[:1]

602    0
Name: Outcome, dtype: int64

In [63]:
y=best_random_grid.predict([[1,124.0,74,36.0,30.5,27.8,0.1,30]])

In [64]:
int(y)

0

##GrideSearch CV

In [65]:
#use best parameters from RandomSeach CV

In [67]:
best_random_grid.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=173, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=900,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)>

In [68]:
rdomsearch.best_params_

{'criterion': 'entropy',
 'max_depth': 173,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 900}

In [69]:
#Gride search CV
from sklearn.model_selection import GridSearchCV

In [71]:
parameter={
    'criterion':[rdomsearch.best_params_['criterion']],
    'max_depth': [rdomsearch.best_params_['max_depth']],
    'max_features': [rdomsearch.best_params_['max_features']],
    'min_samples_leaf': [rdomsearch.best_params_['min_samples_leaf'], 
                         rdomsearch.best_params_['min_samples_leaf']+2, 
                         rdomsearch.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rdomsearch.best_params_['min_samples_split'] - 2,
                          rdomsearch.best_params_['min_samples_split'] - 1,
                          rdomsearch.best_params_['min_samples_split'], 
                          rdomsearch.best_params_['min_samples_split'] +1,
                          rdomsearch.best_params_['min_samples_split'] + 2],
    'n_estimators': [rdomsearch.best_params_['n_estimators'] - 200, rdomsearch.best_params_['n_estimators'] - 100, 
                     rdomsearch.best_params_['n_estimators'], 
                     rdomsearch.best_params_['n_estimators'] + 100, rdomsearch.best_params_['n_estimators'] + 200]

}
print(parameter)

{'criterion': ['entropy'], 'max_depth': [173], 'max_features': ['log2'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [0, 1, 2, 3, 4], 'n_estimators': [700, 800, 900, 1000, 1100]}


In [72]:
gs=GridSearchCV(rf,parameter,n_jobs=-1,cv=3)

In [73]:
gs.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=10,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=2,
                                              min_samples_split=3,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              random_

In [74]:
gs.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=173, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [75]:
gs.best_params_

{'criterion': 'entropy',
 'max_depth': 173,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 700}

In [77]:
y_pred=gs.best_estimator_.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[103   2]
 [  2  47]]
0.9025974025974026
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       105
           1       0.93      0.76      0.83        49

    accuracy                           0.90       154
   macro avg       0.91      0.86      0.88       154
weighted avg       0.90      0.90      0.90       154

